# Gestion de deux sorties numériques d'un Arduino

Ce notebook propose de gérer l'état de deux sorties numériques d'un Arduino : les ports 2 et 3. On pourra connecter une led en série avec sa résistance pour chacune des sorties.

Il vous faudra gérer deux types de codes pour mener à bien ce petit TP :
1. le code Arduino
2. le code python interactif


### 1. Le code source Arduino

Ce code permet de positionner l'état des sorties digitales numéro deux ou trois en fonction de la commande provenant du port série. Cette commande doit être de la forme "channelNumber:channelValue" où channelNumber peut valoir 2 ou 3 et channelValue peut prendre comme valeur 0 pour mettre le port à l'état bas ou tout autre valeur pour mettre le port à l'état haut. On pourra donc avoir les commandes "2:0", "2:1", "3:0" ou "3:1".

Voici le listing du code :

```c
String inputString = ""; // String to hold incoming command
String tempString = ""; // Temporary String to hold type conversion
bool stringComplete = false;  // Whether the string is complete or not
int tokenIndex = 0; // Index of ":" char
int channelNumber, channelValue; // Selected channel (2 or 3) and value
char message[25]; // Sent back message to computer

void setup() {
  // initialize serial:
  Serial.begin(9600);
  
  // Output configuration
  pinMode(2, OUTPUT); // First LED on pin 2
  pinMode(3, OUTPUT); // Second LED on pin 3
}

void loop() {
  // Process command when String is complete
  if (stringComplete) {
    // Detect ":" position in string
    tokenIndex = inputString.indexOf(":");

    // If ":" is detected
    if(tokenIndex != -1) {
      // Compute channel number and value from input string 
      tempString = inputString.substring(0, tokenIndex);
      channelNumber = tempString.toInt();
      tempString = inputString.substring(tokenIndex + 1, inputString.length());
      channelValue = tempString.toInt();
      
      // If channel number is in 2 or 3
      if(channelNumber == 2 or channelNumber == 3) {
        // Switch channel on or off
        if(channelValue == 0) {
          digitalWrite(channelNumber, LOW);
          sprintf(message, "LED on channel %d swithed off", channelNumber);
        } else {
          digitalWrite(channelNumber, HIGH);
          sprintf(message, "LED on channel %d swithed on", channelNumber);
        }
        
        // Print message back to computer
        Serial.println(message);
        
      }
      
    }
    
    // Clear the string:
    inputString = "";
    stringComplete = false;
    
  }
}

/*
  SerialEvent occurs whenever a new data comes in the hardware serial RX. This
  routine is run between each time loop() runs, so using delay inside loop can
  delay response. Multiple bytes of data may be available.
*/
void serialEvent() {

  // While there is a char on serial port
  while (Serial.available()) {
    
    // Read new byte:
    char inChar = (char)Serial.read();
    
    if (inChar == '\n') stringComplete = true; // If new line, string is complete
    else  inputString += inChar; // Else it's new char
      
  }
  
}
```

### 2. Le code python

In [118]:
# Création et paramétrage du port série
import serial
ser = serial.Serial()
ser.baudrate = 9600 
ser.port = '/dev/tty.usbmodem1411' # 'COMx' sous windows Ou '/dev/tty.usbmodem1411' par exemple, sous MacOS et Linux
ser

Serial<id=0x1071fdcf8, open=False>(port='/dev/tty.usbmodem1411', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [119]:
# Ouverture connection au port série
ser.open()
ser.is_open

True

In [120]:
# Création des deux checkbox
import ipywidgets as widgets
led1_checkbox = widgets.Checkbox(value=False, description='LED 1', disabled=False)
led2_checkbox = widgets.Checkbox(value=False, description='LED 2', disabled=False)

In [121]:
# Creation de la fonction interactive
def on_button_checked(b):
    
    state = 0 if(b.owner.value == False) else 1
    
    if(b.owner == led1_checkbox) :
        cmd = "2:" + str(state) + "\n"
    elif(b.owner == led2_checkbox) :
        cmd = "3:" + str(state) + "\n"

    ser.write(cmd.encode())

In [122]:
# Connection des deux checkbox à la fonction
led1_checkbox.observe(on_button_checked)
led2_checkbox.observe(on_button_checked)

In [123]:
# Affichage
display(led1_checkbox)
display(led2_checkbox)

Checkbox(value=False, description='LED 1')

Checkbox(value=False, description='LED 2')